In [1]:
import pandas as pd
import numpy as np
#pretty display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/vcf.txt'
df1 = pd.read_csv(f1, sep='\t')
df1.columns = ['patient', 'vcf']
df1.head(2)

,patient,vcf
0,HTMCP-03-06-02002,/projects/analysis/analysis22/A37160/merge_bwa...
1,HTMCP-03-06-02003,/projects/analysis/analysis22/A37161/merge_bwa...


In [147]:
def process_info(x):
    lst = x.split(';')
    llst = [l.split('=') for l in lst if '=' in l]
    d = {k:v for (k, v) in llst }
    count_split = [int(i) for i in d['DP4'].split(',')]
    ref_count = count_split[0] + count_split[1]
    alt_count = count_split[2] + count_split[3]
    allele_freq = alt_count/(alt_count + ref_count)
    allele_freq = float('{0:.2f}'.format(allele_freq))
    coverage = int(d['DP'])
    return [coverage, ref_count, alt_count, allele_freq]

In [155]:
def parse_vcf(f):
    df = pd.read_csv(f, comment="#", sep='\t')
    df.columns = ['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT', 'patient']
    df = df.drop(['FILTER','patient'], axis=1)
    df['xxx'] = df['INFO'].apply(lambda x: process_info(x))
    df[['coverage', 'ref_count', 'alt_count', 'allele_frequency']] = pd.DataFrame(df.xxx.values.tolist(), index= df.index)
    df = df.drop(['INFO', 'xxx'], axis=1)
    df = df[(df['coverage'] >=10)&(df['allele_frequency']>=.1)]

    of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/snps_test.txt'
    df.to_csv(of, sep='\t', index=False)
    return df

In [159]:
# f = '/projects/analysis/analysis22/A37168/merge_bwa-mem-0.7.6a/125nt/hg19a/A37168_2_lanes_dupsFlagged.varFilter.eff.dbSNP_v137.COSMIC_v64.annotations.vcf'
# df = parse_vcf(f)

In [111]:
# of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/snps_test.txt'
# df.to_csv(of, sep='\t', index=False)

In [161]:
outdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/'
for ix,row in df1.iterrows():
    patient = row['patient']
    vcf = row['vcf']
    df = parse_vcf(vcf)
    of = '{0}{1}.txt'.format(outdir, patient)
    df.to_csv(of, sep='\t', index=False)

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


FileNotFoundError: File b'/projects/analysis/analysis30/A76715/HFFTGCCXY_3/A76715/150nt/hg19a/bwa-mem-0.7.6a-sb/A76715_1_lane_dupsFlagged.varFilter.eff.dbSNP_v137.COSMIC_v64.annotations.vcf' does not exist

In [210]:
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/int_snp.txt'
df2 = pd.read_csv(f2, sep='\t').set_index(['CHR_ID', 'CHR_POS'])
df2.head()

,
CHR_ID,CHR_POS
20,16074337
15,55918731
X,119583024
10,70354622
6,94446700


In [209]:
df2.shape


(2165, 2)

In [213]:

f0 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02330.txt'
df0 = pd.read_csv(f0, sep='\t').set_index(['CHROM', 'POS'])
df0.head(2)

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ID            REF       ALT  QUAL    FORMAT  coverage  ref_count  \
CHROM POS                                                                      
1     10146  .          ACCCC      ACCC  28.0  GT:PL:GQ       188          4   
      10172  .  CCCTAACCTAACC  CCCTAACC  27.5  GT:PL:GQ       125          2   

             alt_count  allele_frequency  
CHROM POS                                 
1     10146         12              0.75  
      10172         14              0.88

In [216]:
df0.reindex(df2.index).dropna(how='all')

,,ID,REF,ALT,QUAL,FORMAT,coverage,ref_count,alt_count,allele_frequency
CHR_ID,CHR_POS,,,,,,,,,


In [218]:
f00 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/patients.txt'
df00 = pd.read_csv(f00, header=None)
df00.head()

,0
0,HTMCP-03-06-02054.txt
1,HTMCP-03-06-02057.txt
2,HTMCP-03-06-02058.txt
3,HTMCP-03-06-02061.txt
4,HTMCP-03-06-02063.txt


In [220]:
for _, row in df00.iterrows():
    f = ''.join(['/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/', row[0]])
    df0 = pd.read_csv(f, sep='\t').set_index(['CHROM', 'POS'])
    df22 = df0.reindex(df2.index).dropna(how='all')
    print('processing {}'.format(f))
    print(df22.shape)
#     if not df22.empty:
#         print(df22.head())

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02054.txt
(0, 9)
processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02057.txt
(0, 9)
processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02058.txt
(0, 9)
processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02061.txt
(0, 9)
processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02063.txt
(0, 9)
processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02068.txt
(0, 9)
processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02070.txt
(0, 9)
processing /projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/germline/HTMCP-03-06-02071.txt
(0, 9)


KeyboardInterrupt: 

In [204]:
df = pd.DataFrame.from_dict({'a':[12,1,3], 'b':[13,4,5], 'c':[22,44,656]})
df.reindex([5, 1, 6])

,a,b,c
5,NaN,NaN,NaN
1,1.0,4.0,44.0
6,NaN,NaN,NaN


In [11]:
lst = [3,8,9,2]
ix = np.argsort(lst)

In [12]:
ix

array([3, 0, 1, 2])

In [13]:
[lst[i] for i in ix]

[2, 3, 8, 9]